In [ ]:
cd ATML-PA-2/

/kaggle/working/ATML-PA-2/IRM


In [3]:
import importlib
import torch
import utils
import architecture
import train
import types
importlib.reload(utils)
importlib.reload(architecture)
importlib.reload(train)


from utils.dataset import FeatureTensorDataset, OfficeHomeDataset, _truncate_resnet_from
from torchvision.models import ResNet50_Weights 
from importlib import reload
import train
from train import train_workflow
import types
from utils.tsne import tsne_plot, get_features
from utils.evaluation import evaluate_ds
import matplotlib.pyplot as plt
import os
from architecture import resnet_classifier
import torch

In [4]:
root_dir = "../datasets/OfficeHomeDataset"
csv_file = f"{root_dir}/ImageInfo.csv"

transform = ResNet50_Weights.IMAGENET1K_V1.transforms()
classes = os.listdir(f"{root_dir}/Art")

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
def make_cache(domains, layer_name):
    model = resnet_classifier(len(classes))
    if len(domains)>1:
        caches = []
        for domain in domains:
            ds = OfficeHomeDataset(root_dir, csv_file, [domain], transform)
            caches.append(FeatureTensorDataset(ds, model, layer_name, device=device, batch_size=64))
        return caches
    else:
        ds = OfficeHomeDataset(root_dir, csv_file, domains, transform)
        return FeatureTensorDataset(ds, model, layer_name, device=device, batch_size=64)


In [11]:
def analysis(sources, target, config, caches_source=None, cache_target=None):
    
    if caches_source is None:
        caches_source = make_cache(sources, 'layer3')
        

    if cache_target is None:
        cache_target = make_cache(target, 'layer3')

    model = resnet_classifier(num_classes=len(classes))
    truncated_model = _truncate_resnet_from(model, 'layer3')

    def _forward_impl(self, x):
        f4 = self.layer4(x)
        x = torch.flatten(self.avgpool(f4), 1)
        x = self.fc(x)
        return x

    import types
    truncated_model.forward = types.MethodType(_forward_impl, truncated_model)

    print("Evaluating on target domain test set")
    
    
    train_workflow(truncated_model, caches_source, cache_target, config, device)
    evaluate_ds(cache_target, truncated_model, device, target[0], None)


In [8]:
sources = ["Art", "Real World", "Product"]
target = ["Clipart"]

caches_source = make_cache(sources, 'layer3')
cache_target = make_cache(target, 'layer3')

Precomputing up to layer3: 100%|██████████| 69/69 [00:46<00:00,  1.48it/s]


In [ ]:
config = {'epochs':25, 
          'lr':1e-3, 
          'batch_size':128, 
          'weight_decay':1e-5, 
          'phi':1.0,
          'scheduling_constant':2.0,
          }
analysis(sources, target, config, caches_source, cache_target)

Evaluating on target domain test set


Training:   0%|          | 0/12 [00:00<?, ?it/s]


Epoch 1/12


Training:   8%|▊         | 1/12 [00:24<04:32, 24.74s/it]

	ERM loss: 11.938709	Grad penalty: 0.015282	Train Accuracy: 17.18	Test Accuracy: 6.85

Epoch 2/12


...

	ERM loss: 7.741431	Grad penalty: 0.255357	Train Accuracy: 38.36	Test Accuracy: 21.24

Epoch 9/12


Training:  75%|███████▌  | 9/12 [03:42<01:14, 24.87s/it]

	ERM loss: 7.352030	Grad penalty: 0.220212	Train Accuracy: 44.41	Test Accuracy: 24.79

Epoch 10/12


Training:  83%|████████▎ | 10/12 [04:07<00:49, 24.85s/it]

	ERM loss: 6.863207	Grad penalty: 0.242195	Train Accuracy: 46.82	Test Accuracy: 25.84

Epoch 11/12


In [19]:
classes[5]

'Bucket'